Antoni Pawłowicz 191733

<div align="center">
<em>
METODY INTERAKCJI CZŁOWIEK MASZYNA</br></br>
Laboratoria 2</br>
Algorytmy estymacji i detekcji postawy ciała
</em>

In [ ]:
import cv2
import numpy as np

### _Zadanie 1_
Używając wcześniej zdefiniowanych par połącz odpowiednie punkty liniami
tworząc szkielet obiektu.

In [ ]:
protoTxt = "OpenPose/pose_deploy_linevec_faster_4_stages.prototxt"
wagi = "OpenPose/pose_iter_160000.caffemodel"
cam_id = '/home/anthonyp57/VSCode_projects/interakcja/lab_3/man_vid.mp4'

net = cv2.dnn.readNetFromCaffe(protoTxt, wagi)

cap = cv2.VideoCapture(cam_id)

part_to_idx = {"Head": 0, "Neck": 1, "RShoulder": 2, "RElbow": 3, "RWrist": 4,
"LShoulder": 5, "LElbow": 6, "LWrist": 7, "RHip": 8, "RKnee": 9,
"RAnkle": 10, "LHip": 11, "LKnee": 12, "LAnkle": 13, "Chest": 14,
"Background": 15}

BODY_PAIRS = [ ["Head", "Neck"], ["Neck", "RShoulder"], ["RShoulder",
"RElbow"],["RElbow", "RWrist"], ["Neck", "LShoulder"], ["LShoulder",
"LElbow"],["LElbow", "LWrist"], ["Neck", "Chest"], ["Chest", "RHip"],
["RHip", "RKnee"],["RKnee", "RAnkle"], ["Chest", "LHip"], ["LHip",
"LKnee"], ["LKnee", "LAnkle"] ]

pairs_idx = [(part_to_idx[a[0]], part_to_idx[a[1]]) for a in BODY_PAIRS]
part_indices = part_to_idx.values()[:-1] #no backgound
part_names = part_to_idx.keys()[:-1]

res, img = cap.read()
if not res:
    raise IOError("webcam failure")

limb_pos = {i:None for i in part_indices}

inWidth = img.shape[1]
inHeight = img.shape[0]

inpBlob = cv2.dnn.blobFromImage(img, 1.0 / 255, (inWidth,
inHeight), (0, 0, 0), swapRB=False, crop=False)

# ustawienie jako input
net.setInput(inpBlob)
#predykcja
output = net.forward()

limb_map = output[0,:len(part_indices),:,:]

for i,l in enumerate(limb_map):

    probMap = cv2.resize(probMap, (inWidth, inHeight))
    # zamiana skali kolorów dla confidence map i oznaczenie kolorem czerwonym
    probMap2 = cv2.cvtColor(probMap, cv2.COLOR_GRAY2RGB) #part prob
    probMap2[:,:,0]=0
    probMap2[:,:,1]=0

    frame4=cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame5 = cv2.addWeighted(frame4, 0.4, probMap2, 0.6, 0, dtype=cv2.CV_32F).astype(np.uint8)

    minVal, prob, minLoc, point = cv2.minMaxLoc(probMap)

    if prob > threshold :
        x,y = point
        x,y = int(x),int(y)
        cv2.circle(frame2, (x,y), 5, (255, 255, 255), thickness=2, lineType=cv2.FILLED)
        # cv2.putText(frame2, "{}".format(i), (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 3, lineType=cv2.LINE_AA)
        cv2.putText(frame2, f'{part_names[i]}', (x-5, y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 3, lineType=cv2.LINE_AA)
        limb_pos[i] = (x,y)
    else :
        limb_pos[i] = None

#draw lines between connecting body parts
for p in pairs_idx:
    cv2.line(frame, limb_pos[p[0], limb_pos[p[1]]], 3)

cv2.imshow('posenet', frame)

### _Zadanie 2_
 Stwórz algorytm detekcji dowolnej pozy ciała na podstawie wzajemnego
położenia punktów kluczowych (ich wzajemnych odległości, kątów pomiędzy
połączeniami par itp.). Dokładnie opisz działanie algorytmu i przedstaw wyniki jego
działania.

In [ ]:
def angle_between_3_vectors(x, y, z):
    shoulder = np.array(shoulder)
    elbow = np.array(elbow)
    wrist = np.array(wrist)

    shoulder_elbow = elbow - shoulder
    shoulder_elbow /= np.linalg.norm(shoulder_elbow) #normalize by norm to = 1

    shoulder_wrist = wrist - shoulder
    shoulder_wrist /= np.linalg.norm(shoulder_wrist)

    angle = np.arccos(np.clip(np.dot(shoulder_elbow, shoulder_wrist), -1.0, 1.0)) #cos similarity inverse so to speak, clip for stability
    return np.degrees(angle) #angle in degrees